# RSA analysis of the Neuropixel Data

This notebook analyzes Allen Institute's Neuropixel data. It focuses on the `functional_connectivity` dataset since that dataset seems to be richer (more repeats of the same stimuli), with more rest time in-between sessions, and has better quality data (only one animal missing pupilometry measurements).

In [1]:
from src.neuropixel import Neuropixel

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
HOME = Path('/media/internal/projects/Allen/Ecephys')
npx = Neuropixel(HOME)

In [3]:
data_type = 'functional_connectivity'

## Natural Movie 1

This movie is played in both sessions of `functional_connectivity`, each repeated 60 times on each session (there are also 20 repeats with temporal shuffle that I'm going to ignore for now). The movie has 30s duration, so I will extract 30 "blocks" of 1s duration.

Let's start by getting the dataframes for each of these blocks.

In [4]:
table = npx.cache.get_session_table()
session_ids = table.loc[table.session_type == data_type].index.values

In [5]:
# remove sessions w/o eye data
session_ids = np.delete(session_ids, np.where(session_ids == 768515987))
session_ids = np.delete(session_ids, np.where(session_ids == 840012044))

In [6]:
session_ids

array([766640955, 767871931, 771160300, 771990200, 774875821, 778240327,
       778998620, 779839471, 781842082, 786091066, 787025148, 789848216,
       793224716, 794812542, 816200189, 819186360, 819701982, 821695405,
       829720705, 831882777, 835479236, 839068429, 839557629, 847657808],
      dtype=int64)

In [7]:
movie_name = "natural_movie_one_more_repeats"
rois = ["VISrl", "VISp", "VISpm", "VISam", "VISal", "VISl"]
# rois = ["VISp"]
seconds = np.arange(0, 30)

In [8]:
# Read all the csvs
# get files from all subs
files = npx.SAVE_DIR.glob('*.csv')
data_files = sorted(list(files))  # sort and list the files

dfs = []
for f in data_files:
    df = pd.read_csv(f, index_col=0)
    df["session_id"] = str(f.name).split('_')[0].split('-')[1]
    df["movie_sec"] = df["movie_sec"] + 1
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
# load into one dataframe
# df = pd.concat(map(lambda f: pd.read_csv(f, index_col=0), data_files), ignore_index=True)

In [9]:
df

,stimulus_presentation_id,unit_id,start_time,stop_time,stimulus_block,stimulus_name,stimulus_condition_id,spike_counts,roi,movie_sec,median_speed,mean_speed,pupil_area,session_id
0,5209,950943078,2326.237147,2326.270507,3.0,natural_movie_one_more_repeats,1152,0,VISal,1,1.729311,2.009071,0.001445,766640955
1,5209,950943108,2326.237147,2326.270507,3.0,natural_movie_one_more_repeats,1152,3,VISal,1,1.729311,2.009071,0.001445,766640955
2,5209,950943186,2326.237147,2326.270507,3.0,natural_movie_one_more_repeats,1152,0,VISal,1,1.729311,2.009071,0.001445,766640955
3,5209,950943208,2326.237147,2326.270507,3.0,natural_movie_one_more_repeats,1152,13,VISal,1,1.729311,2.009071,0.001445,766640955
4,5209,950943245,2326.237147,2326.270507,3.0,natural_movie_one_more_repeats,1152,3,VISal,1,1.729311,2.009071,0.001445,766640955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971995,76251,950954680,8279.241058,8279.274419,8.0,natural_movie_one_more_repeats,492,1,VISrl,9,1.073632,1.174236,0.001474,767871931
971996,76251,950955121,8279.241058,8279.274419,8.0,natural_movie_one_more_repeats,492,1,VISrl,9,1.073632,1.174236,0.001474,767871931
971997,76251,950955147,8279.241058,8279.274419,8.0,natural_movie_one_more_repeats,492,0,VISrl,9,1.073632,1.174236,0.001474,767871931
971998,76251,950955160,8279.241058,8279.274419,8.0,natural_movie_one_more_repeats,492,0,VISrl,9,1.073632,1.174236,0.001474,767871931


In [12]:
# create population vector: (n_units, block)
df.groupby(["session_id", "roi", "stimulus_block", "stimulus_condition_id", "unit_id", "movie_sec"], as_index=False)["spike_counts", "median_speed", "pupil_area"].mean()

C:\Users\shari\miniconda3\envs\neuro\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,session_id,roi,stimulus_block,stimulus_condition_id,unit_id,movie_sec,spike_counts,median_speed,pupil_area
0,766640955,VISal,3.0,282,950943078,2,1.000000,1.288191,0.001187
1,766640955,VISal,3.0,282,950943108,2,7.166667,1.288191,0.001187
2,766640955,VISal,3.0,282,950943186,2,0.500000,1.288191,0.001187
3,766640955,VISal,3.0,282,950943208,2,14.900000,1.288191,0.001187
4,766640955,VISal,3.0,282,950943245,2,3.833333,1.288191,0.001187
...,...,...,...,...,...,...,...,...,...
32395,767871931,VISrl,8.0,1152,950954680,1,6.000000,5.276708,0.001708
32396,767871931,VISrl,8.0,1152,950955121,1,3.266667,5.276708,0.001708
32397,767871931,VISrl,8.0,1152,950955147,1,0.266667,5.276708,0.001708
32398,767871931,VISrl,8.0,1152,950955160,1,0.500000,5.276708,0.001708


In [ ]:
sessions = df.session_id.unique()
rois = df.roi.unique()
block = df.stimulus_block.unique()
repeat = stimulus
for sess in